In [1]:
import numpy as np
from twitter_scraper import get_tweets
from twitterscraper import query_tweets
import re
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
person_of_interest = 'realDonaldTrump'
tweets_info = query_tweets('from:' + person_of_interest, begindate=dt.date(2018,4,11), enddate=dt.date.today())
#     # def query_tweets(query, limit=None, begindate=dt.date(2017,1,1), enddate=dt.date.today(), poolsize=20, lang=''):
    
# tweets = []
# for tweet_info in tweets_info:
#     tweet = tweet_info.text
#     tweets.append(tweet)

# tweets = np.array(tweets)

# print(tweets)

In [3]:
# tweets_info = get_tweets(person_of_interest, pages = 10)
tweets = []
for tweet_info in list(tweets_info):
#     text = tweet_info['text']
    text = tweet_info.text #Figure out which one to use. It does not work consistently yet
    # Add NEWTWEET to help recognize starting word of tweet for future processing
    tweets.append("NEWTWEET " + text)
    
tweets = np.array(tweets)
num_tweets = len(tweets)
print("Number of tweets: " + str(num_tweets))
# print(tweets)

Number of tweets: 36


In [4]:
words = []
for tweet in tweets:
    words.extend(tweet.split(" "))
    
cleaned_words = []
for word in words:
    if ".com" not in word and "www." not in word and "http" not in word and word != '':
        cleaned_words.append(word)
        
words = cleaned_words
cleaned_words_no_repeat = [words[0]] # could be repeated 'NEWTWEET' due to deleting URL's
        
for i in range(1, len(words)):
    if not (words[i] == "NEWTWEET" and words[i] == words[i-1]):
        cleaned_words_no_repeat.append(words[i])
        
words = cleaned_words_no_repeat
words = np.array(words)
# print(words)

In [5]:
G = nx.DiGraph()
# Initialize states with start and end states (assuming no word is 'S' or 'E')
states = ['S', 'E']
for word in words:
    if word not in states and word != "" and word != "NEWTWEET":
        states.append(word)
G.add_nodes_from(states)
# nx.draw(G, with_labels = True)

In [6]:
freq = {}
edges = []

# Edge case for first word in tweet not being connected to Start state

for i in range(len(words) - 1):
    if words[i] == "NEWTWEET":
        edges.append(('S', words[i + 1], 1))
        continue
    elif words[i+1] == "NEWTWEET":
        edges.append((words[i], 'E', 1))
        continue
    elif (words[i], words[i+1]) in freq:
        freq[(words[i], words[i+1])] += 1
    else:
        freq[(words[i], words[i+1])] = 1
        
    if words[i].endswith(('.', '!', '?')):
#         print(words[i], words[i+1])
        edges.append((words[i], 'E', 1))
        edges.append(('S', words[i + 1], 1))
        # remove 'word. Word' so no edge is made between them?
        
edges.append(('E', 'S', 1))

for edge in freq:
    edges.append((edge[0], edge[1], freq[edge]))
    
G.add_weighted_edges_from(edges)
# nx.draw(G, with_labels = True)
# nx.draw(G)

In [7]:
# print(list(G.successors('S')))
# print(list(G.predecessors('E')))

# print(list(G.predecessors('S')))
# print(list(G.successors('E')))

In [8]:
# Creates numpy adjacency matrix of states, where Starting node is row 0 and Ending node is row 1
matrix = nx.to_numpy_matrix(G, nodelist = states)
matrix = np.array(matrix).tolist() #DON'T DO THIS?
# print(matrix)
# print(np.array(matrix))

In [9]:
# Sanity check to see if adjacency matrix reflects actual state of MC
num = 0
for i in range(len(matrix[0])):
    num += matrix[0][i]
print(num == len(list(G.successors('S')))) 

True


In [10]:
for i in range(len(matrix)):
    if np.array(matrix[i]).sum(axis=0) != 0:
        continue
    else:
        print(i)

matrix = np.delete(matrix, len(matrix)-1, axis=1)
matrix = np.delete(matrix, len(matrix)-1, axis=0)

716


In [11]:
matrix = np.array(matrix)

#The fact that I can not use this is signaling to me that something is wrong.
#Perhaps there are nodes that do not have a next to jump to. This would make NO sense?
transition_mat = np.array(matrix)/np.array(matrix).sum(axis=1)[:,None]

def get_next_state(i, transition_matrix):
    return np.random.choice(transition_matrix.shape[1], 1, p=transition_matrix[i])[0]

def perform_random_walk(s, e, transition_matrix):
    curr_state = s
    l = [curr_state]
    while curr_state != e:
        curr_state = get_next_state(curr_state, transition_matrix)
        l.append(curr_state)
    return np.array(l)

def generate_sentence():
#     print(''.join([edges[i][0] + ' ' for i in perform_random_walk(0, 1, transition_mat) if edges[i][0] is not 'S']))
    print([edges[i][0] + ' ' for i in perform_random_walk(0, 1, transition_mat) if edges[i][0] is not 'S'])
    print([edges[i][0] + ' ' for i in perform_random_walk(0, 1, transition_mat)])    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [12]:
n = 25
for i in range(n):
    generate_sentence()

['Holocaust. ', 'LIED! ', 'LIED! ', 'increases ', 'as ', 'newspaper! ', 'who ', 'make ', 'this ', 'LIED! ', 'today. ', 'same ', 'point. ', 'With ', 'the ', 'of ', 'Would ', 'and ', 'disaster. ']
['S ', 'my ', 'Administration, ', 'letter). ', 'thieves ', 'and ', 'We ', 'LIED! ', 'made ', 'up ', 'by ', 'this ', 'den ', 'as ', 'one ', 'the ', 'disaster. ']
['it. ', 'They ', 'will ', 'lowlifes! ', 'him. ', 'newspaper! ', 'LIED! ', 'a ', 'and..... ', 'LIED! ', 'He ', 'leaked ', 'CLASSIFIED ', 'information, ', 'collusion, ', 'as ', 'one ', 'the ', 'disaster. ']
['S ', 'Slimeball! ', 'LIED! ', 'are ', 'lowlifes! ', 'work ', 'believe! ', 'S ', 'had. ', 'S ', 'S ', 'today. ', 'S ', 'you ', 'cherish ', 'disaster. ']
['cooperative, ', 'disaster. ']
['S ', 'Ty ', 'the ', 'Cobb, ', 'my ', 'Special ', 'Counsel, ', 'disaster. ']
['FBI. ', '“botch ', 'America, ', 'Just ', 'are ', 'doing ', 'newspaper! ', 'LIED! ', 'an ', 'end ', 'Cities. ', 'We ', 'already ', 'have ', 'TPP ', 'newspaper! ', 'handling 